In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
71,ABW,Aruba,2020-06-02,101,0,3,0,945.994,0.0,28.099,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
216,AFG,Afghanistan,2020-06-02,15750,545,265,8,404.590,14.0,6.807,...,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5
289,AGO,Angola,2020-06-02,86,0,4,0,2.617,0.0,0.122,...,2.405,1.362,5819.495,NaN,276.045,3.94,NaN,NaN,26.664,NaN


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-01,France,153238.0,0.0,28859.0,0.0,0.0,0.0
0,2020-06-01,China,84215.0,0.0,4653.0,0.0,0.0,0.0
0,2020-06-01,Italy,233552.0,0.0,33542.0,0.0,0.0,0.0
0,2020-06-01,Spain,240105.0,0.0,27147.0,0.0,0.0,0.0
0,2020-06-01,United States,1815465.0,0.0,105457.0,0.0,0.0,0.0
0,2020-06-01,World,6273925.0,0.0,376387.0,0.0,0.0,0.0
0,2020-06-01,United Kingdom,276769.0,0.0,38743.0,0.0,0.0,0.0
0,2020-06-01,Germany,182184.0,0.0,8520.0,0.0,0.0,0.0
0,2020-06-01,Iran,153822.0,0.0,7860.0,0.0,0.0,0.0
0,2020-06-01,Turkey,164878.0,0.0,4564.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-02,France,153238.0,152091.0,28859.0,28833.0,1147.0,26.0
0,2020-06-02,China,84215.0,84154.0,4653.0,4638.0,61.0,15.0
0,2020-06-02,Italy,233552.0,233197.0,33542.0,33475.0,355.0,67.0
0,2020-06-01,Spain,240105.0,239638.0,27147.0,27940.0,467.0,-793.0
0,2020-06-02,United States,1815465.0,1811277.0,105457.0,105147.0,4188.0,310.0
0,2020-06-02,World,6273925.0,6245352.0,376387.0,376427.0,28573.0,-40.0
0,2020-06-02,United Kingdom,276769.0,276332.0,38743.0,39045.0,437.0,-302.0
0,2020-06-02,Germany,182184.0,182028.0,8520.0,8522.0,156.0,-2.0
0,2020-06-02,Iran,153822.0,154445.0,7860.0,7878.0,-623.0,-18.0
0,2020-06-02,Turkey,164878.0,164769.0,4564.0,4563.0,109.0,1.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")